In [9]:
import json
import subprocess #Need to clean up GPU memory after execution so forced to restart interpreter
import numpy as np

In [10]:
def find_omegas(dat_lines, start_idx, end_idx, entry_line):
    METADATA_LINE_LEN = 51 
    SCAN_ID_IDX = 7
    OMEGA_IDX = 31

    omegas = np.zeros(end_idx - start_idx + 1)
    found_omegas = np.zeros(end_idx - start_idx + 1)

    for i in range(entry_line, 0, -1):
        line = dat_lines[i].split(' ')
        if len(line) == METADATA_LINE_LEN:
            scan_num = int(line[SCAN_ID_IDX])

            if scan_num < start_idx:
                break

            if scan_num <= end_idx:
                store_idx = scan_num - start_idx
                omegas[store_idx] = float(line[OMEGA_IDX])
                found_omegas[store_idx] = True

    if not np.all(found_omegas):
        raise ValueError(f"Unable to find all omegas for given scan range {start_idx}-{end_idx}")    
    
    return omegas

In [11]:
scans = []
NUM_FIELD_BEGIN = 10
NUM_FIELD_END = 20


with open('working_dir/faber_sam1_s12_crack_tomo_TomoFastScan.dat') as dat_f:
    read_counter = 0
    cur_scan = {}
    dat_lines = dat_f.readlines()

    for i, line in enumerate(dat_lines):
        if line.startswith('End'):
            read_counter = 2
        elif read_counter == 2:
            read_counter -= 1
            cur_scan['start'] = int(line.split(' ')[4].strip()) + NUM_FIELD_BEGIN
        elif read_counter == 1:
            read_counter -= 1
            cur_scan['end'] = int(line.split(' ')[4].strip()) - NUM_FIELD_END
            cur_scan['omega'] = find_omegas(dat_lines, 
                                             cur_scan['start'],
                                             cur_scan['end'],
                                             i)
            scans.append(cur_scan)
            cur_scan = {}
            

In [12]:
with open('working_dir/config.json', 'r') as config_f:
    config = json.load(config_f)

In [13]:
for scan in scans:
    print(f"{scan['start'], scan['end']}")

(133706, 135506)
(135537, 137337)
(137368, 139168)
(139200, 141000)
(141031, 142831)
(142862, 144662)
(144694, 146494)
(146525, 148325)
(148356, 150156)
(150188, 151988)
(152019, 153819)
(153850, 155650)
(155682, 157482)
(157513, 159313)
(159344, 161144)
(161177, 162977)
(163008, 164808)
(164839, 166639)
(166670, 168470)
(168501, 170301)
(170332, 172132)
(172163, 173963)
(173994, 175794)


In [18]:
OUT_DIR = 'working_dir/voids_remap' 

error_scans = []
for i, scan in enumerate(scans):
    config['img_range'] = [scan['start'], scan['end']]
    config['omega'] = scan['omega'].tolist()
    assert scan['end'] - scan['start'] + 1 == len(config['omega'])

    curr_config_fp = 'working_dir/temp_all_config.json'
    with open(curr_config_fp, 'w') as curr_conf_f:
        json.dump(config, curr_conf_f)
    map_proc = subprocess.run(['python', 'coarse_mesh.py', f'{curr_config_fp}', OUT_DIR])
    if map_proc.returncode != 0:
        error_scans.append(f"{scan['start']}-{scan['end']}")

    print(f'Finished scan {i + 1}/{len(scans)}')

2022-09-26 10:44:32.975851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 10:44:33.320708: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 10:44:36.710586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 10:44:36.710811: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

	TIME: COARSE RECON 6.29 secs
	STAT: voids found - 34033
	TIME: LABELING 24.40 secs
void2mesh_mproc: preserve_feature : False, edge_thres : 0, b : 4, nprocs : None
	TIME: compute void mesh 7.50 secs
Finished scan 1/23


2022-09-26 10:47:39.250082: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 10:47:39.531873: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 10:47:42.229365: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 10:47:42.230574: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

	TIME: COARSE RECON 6.47 secs
	STAT: voids found - 27874
	TIME: LABELING 26.05 secs
void2mesh_mproc: preserve_feature : False, edge_thres : 0, b : 4, nprocs : None
	TIME: compute void mesh 7.09 secs
Finished scan 2/23


2022-09-26 10:50:43.900551: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 10:50:44.177115: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 10:50:47.938996: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 10:50:47.940177: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

...

Finished scan 21/23


2022-09-26 11:34:32.958944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 11:34:33.256941: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 11:34:36.237308: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 11:34:36.237517: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Finished scan 22/23


2022-09-26 11:34:48.746438: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 11:34:49.090169: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 11:34:51.922048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 11:34:51.922242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Finished scan 23/23
